In [1]:
from scqbf.scqbf_evaluator import *
from scqbf.scqbf_solution import *
from scqbf.scqbf_ts import *
from scqbf.scqbf_instance import *

import pandas as pd
import numpy as np
import pickle

In [2]:
instance_paths = [(f"instances/gen{i}/instance{j}.txt", i, j) for i in range(1, 4) for j in range(1, 6)]

In [3]:
alpha_1 = 0.3   # More greedy
alpha_2 = 0.8   # More random

tenure_1 = lambda x: math.ceil(math.sqrt(x))
tenure_2 = lambda x: min(math.ceil(x/4), 20)

In [ ]:
def run_experiment(config: dict, tenure_func) -> pd.DataFrame:
    results = []
    exp_num = 1

    for instance_path, gen, inst in instance_paths:
        instance = read_max_sc_qbf_instance(instance_path)
        print(f"{exp_num}: {inst}th instance of generation strategy {gen}. Path: {instance_path}")
        exp_num += 1
        
        time_limit = 60*30
        tenure = tenure_func(instance.n)
        tabu_search = ScQbfTabuSearch(instance, config=config, max_iterations=None, time_limit_secs=time_limit, tenure=tenure, patience=1000)
        best_solution = tabu_search.solve()

        if tabu_search.solve_time >= time_limit:
            stop_reason = "time_limit"
        else:
            stop_reason = "patience_exceeded" 
        
        evaluator = ScQbfEvaluator(instance)
        
        results.append({
            'gen': gen,
            'inst': inst,
            'n': instance.n,
            'stop_reason': stop_reason,
            'best_objective': evaluator.evaluate_objfun(best_solution),
            'coverage': evaluator.evaluate_coverage(best_solution),
            'time_taken': tabu_search.solve_time
        })
        
        last_result = results[-1]
        print(f"\tBest objective value: {last_result['best_objective']:.4f}")
        print(f"Selected elements: {best_solution.elements}")
        print(f"\tCoverage: {last_result['coverage']:.2%}")
        print(f"\tTime taken (secs): {last_result['time_taken']:.4f} s")
        print(f"\tStop reason: {last_result['stop_reason']}")
        print()
    
    df = pd.DataFrame(results)
    return df

## Configuração 1 - PADRÃO:
GRASP com parâmetro α1 , first-improving e heurística construtiva padrão.

In [5]:
config_1 = {
    "construction_method": "traditional",
    "construction_args": [],
    "local_search_method": "first_improve",
}
results_df = run_experiment(config_1, tenure_1)

with open("pickles/results_config_1.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
Time limit of 10 seconds reached, stopping Tabu Search.
	Best objective value: 4515.4400
Selected elements: [161, 41, 126, 159, 187, 142, 178, 38, 112, 65, 37, 123, 114, 54, 71, 87, 189, 150, 113, 171, 67, 105, 40, 157, 147, 92, 78, 160, 28, 133, 144, 25, 48, 172, 12, 91, 166, 5, 79, 175, 32, 76, 9, 177, 98, 158, 68, 36, 169, 80, 154, 102, 15, 192, 46, 110, 47, 27, 94, 90, 131, 146, 83, 44, 124, 119, 99, 11, 97, 153, 43, 69, 85, 167, 125, 14, 163, 50, 22, 137, 174, 139, 136, 182, 53, 116, 179, 168, 111, 129, 1, 148, 60, 135, 23, 89, 72, 127, 145, 173, 196, 118, 109, 115, 132, 3, 164, 197, 64, 151, 101, 10, 198, 195]
	Coverage: 100.00%
	Time taken (secs): 10.0955 s
	Stop reason: time_limit

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt


KeyboardInterrupt: 

## Configuração 2 - PADRÃO+ALPHA:
GRASP PADRÃO mas com parâmetro α2 .

In [ ]:
config_2 = {
    "construction_method": "traditional",
    "construction_args": [alpha_2],
    "local_search_method": "first_improve",
}
results_df = run_experiment(config_2, tenure_func=tenure_2)

with open("pickles/results_config_2.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt


TypeError: ScQbfTabuSearch.__init__() got an unexpected keyword argument 'tenure'

## Configuração 3 - PADRÃO+BEST:
GRASP PADRÃO mas com best-improving.

In [ ]:
config_3 = {
    "construction_method": "traditional",
    "construction_args": [alpha_1],
    "local_search_method": "best_improve",
}
results_df = run_experiment(config_3)

with open("pickles/results_config_3.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
Time limit of 1800 seconds reached, stopping GRASP.
	Best objective value: 4614.2500
Selected elements: [69, 147, 105, 196, 158, 50, 92, 11, 9, 125, 192, 32, 90, 22, 5, 46, 71, 15, 171, 78, 114, 154, 118, 111, 112, 65, 48, 172, 12, 124, 113, 197, 14, 117, 174, 161, 41, 163, 54, 123, 178, 102, 189, 98, 40, 38, 126, 129, 166, 28, 91, 85, 148, 128, 169, 116, 43, 160, 76, 27, 159, 25, 142, 144, 36, 150, 157, 79, 72, 187, 110, 67, 47, 99, 37, 60, 179, 68, 195, 146, 131, 136, 133, 135, 80, 175, 97, 153, 167, 87, 44, 173, 33, 198, 73, 64, 115, 182, 51, 109, 89, 145, 83, 53, 61, 137, 139, 94, 23, 151, 10, 8, 1, 134, 45, 86, 81, 13, 184, 152, 93]
	Coverage: 100.00%
	Time taken (secs): 1801.0888 s
	Stop reason: time_limit

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt
Time limit of 1800 seconds reached, stopping GRASP.
	Best objective value: 4668.0700
Selected elements: [177, 76, 57, 88, 1

,gen,inst,n,stop_reason,best_objective,coverage,time_taken
0,1,1,200,time_limit,4614.25,1.0,1801.088770
1,1,2,200,time_limit,4668.07,1.0,1805.202337
2,1,3,50,patience_exceeded,645.62,1.0,82.677780
3,1,4,25,patience_exceeded,261.06,1.0,5.137597
4,1,5,100,patience_exceeded,1525.48,1.0,857.760222
5,2,1,50,patience_exceeded,575.87,1.0,47.580676
6,2,2,400,time_limit,14339.26,1.0,1871.485279
7,2,3,25,patience_exceeded,210.16,1.0,5.297815
8,2,4,400,time_limit,13536.58,1.0,1865.697200
9,2,5,100,patience_exceeded,1550.49,1.0,760.455492


## Configuração 4 - PADRÃO+HC1:
GRASP PADRÃO mas com método de construção alternativo 1.

In [ ]:
p_1 = 0.2
config_4 = {
    "construction_method": "random_plus_greedy",
    "construction_args": [alpha_1, p_1],
    "local_search_method": "first_improve",
}
results_df = run_experiment(config_4)

with open("pickles/results_config_4.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
Patience exhausted, no improvement to the objective solution in 1000 iterations, stopping GRASP.
	Best objective value: 4614.2500
Selected elements: [178, 94, 27, 133, 148, 117, 68, 159, 11, 86, 28, 115, 145, 85, 102, 53, 114, 33, 167, 36, 171, 5, 8, 10, 12, 14, 15, 23, 32, 37, 41, 43, 46, 47, 40, 38, 48, 44, 61, 64, 65, 69, 71, 45, 13, 72, 54, 51, 73, 76, 78, 79, 80, 81, 83, 87, 90, 91, 92, 97, 105, 22, 9, 110, 111, 113, 116, 118, 67, 109, 25, 112, 123, 126, 128, 129, 131, 134, 135, 136, 137, 139, 142, 144, 146, 150, 147, 152, 151, 154, 60, 158, 157, 1, 160, 161, 163, 169, 172, 173, 174, 175, 179, 182, 187, 189, 192, 195, 196, 198, 50, 89, 166, 98, 99, 153, 184, 125, 197, 124, 93]
	Coverage: 100.00%
	Time taken (secs): 486.8412 s
	Stop reason: patience_exceeded

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt
Patience exhausted, no improvement to the objective solution in 1000 ite

,gen,inst,n,stop_reason,best_objective,coverage,time_taken
0,1,1,200,patience_exceeded,4614.25,1.0,486.841180
1,1,2,200,patience_exceeded,4668.07,1.0,682.984163
2,1,3,50,patience_exceeded,645.62,1.0,5.257797
3,1,4,25,patience_exceeded,261.06,1.0,1.545747
4,1,5,100,patience_exceeded,1525.48,1.0,70.368411
5,2,1,50,patience_exceeded,575.87,1.0,11.662227
6,2,2,400,time_limit,14472.64,1.0,1800.175431
7,2,3,25,patience_exceeded,210.16,1.0,0.951277
8,2,4,400,time_limit,13589.31,1.0,1802.411006
9,2,5,100,patience_exceeded,1550.49,1.0,72.596032


## Configuração 5 - PADRÃO+HC2:
GRASP PADRÃO mas com método de construção alternativo 2.

In [ ]:
p_2 = 0.1
config_5 = {
    "construction_method": "sampled_greedy",
    "construction_args": [alpha_1, p_1],
    "local_search_method": "first_improve",
}
results_df = run_experiment(config_5)

with open("pickles/results_config_5.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
Patience exhausted, no improvement to the objective solution in 1000 iterations, stopping GRASP.
	Best objective value: 4614.2500
Selected elements: [43, 179, 48, 27, 80, 1, 10, 11, 12, 13, 14, 22, 9, 32, 28, 33, 25, 23, 37, 38, 40, 44, 8, 41, 45, 46, 47, 36, 53, 5, 64, 65, 68, 69, 71, 50, 72, 60, 73, 76, 79, 81, 86, 61, 87, 90, 91, 92, 94, 15, 97, 98, 105, 110, 54, 109, 111, 78, 113, 114, 67, 115, 51, 116, 117, 118, 112, 123, 83, 126, 128, 99, 129, 131, 102, 133, 136, 137, 135, 142, 89, 144, 139, 145, 146, 148, 150, 151, 153, 154, 157, 158, 159, 160, 161, 163, 167, 169, 172, 152, 173, 175, 178, 182, 174, 187, 189, 192, 195, 171, 196, 147, 197, 198, 184, 85, 166, 134, 125, 124, 93]
	Coverage: 100.00%
	Time taken (secs): 581.4256 s
	Stop reason: patience_exceeded

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt
Patience exhausted, no improvement to the objective solution in 1000 ite

,gen,inst,n,stop_reason,best_objective,coverage,time_taken
0,1,1,200,patience_exceeded,4614.25,1.0,581.425603
1,1,2,200,patience_exceeded,4668.07,1.0,318.423466
2,1,3,50,patience_exceeded,645.62,1.0,7.333413
3,1,4,25,patience_exceeded,261.06,1.0,1.157546
4,1,5,100,patience_exceeded,1525.35,1.0,50.110539
5,2,1,50,patience_exceeded,575.87,1.0,9.208786
6,2,2,400,time_limit,14469.84,1.0,1801.054240
7,2,3,25,patience_exceeded,210.16,1.0,0.969585
8,2,4,400,time_limit,13589.31,1.0,1800.109128
9,2,5,100,patience_exceeded,1550.49,1.0,88.714512
